##### Authors: Nay Lin
This notebook wants to test MBart before and after adding the self-consistency layer.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd

# Load the data
train_df = pd.read_csv('/content/drive/MyDrive/data/train.csv')
test_df = pd.read_csv('/content/drive/MyDrive/data/test.csv')
validation_df = pd.read_csv('/content/drive/MyDrive/data/validation.csv')

# Sanity check
print(validation_df['en'])
print(validation_df['zh'])

0      Last year I showed these two slides so that  d...
1      But this understates the seriousness of this p...
2      The arctic ice cap is, in a sense,  the beatin...
3          It expands in winter and contracts in summer.
4      The next slide I show you will be  a rapid fas...
                             ...                        
874    You increase paralysis, and you decrease satis...
875                          Everybody needs a fishbowl.
876    This one is almost certainly too limited --  p...
877    But the absence of some metaphorical fishbowl ...
878                                 Thank you very much.
Name: en, Length: 879, dtype: object
0      去年我给各位展示了两个 关于北极冰帽的演示 在过去三百万年中 其面积由相当于美国南方48州面...
1                        但这些没能完全说明这个问题的严重性 因为这没有表示出冰帽的厚度
2                               感觉上，北极冰帽 就好象全球气候系统中跳动的心脏
3                                          冬天心脏舒张，夏天心脏收缩
4                                  下面我要展示的是 在过去25年里的极剧变化
                             ...                   

In [13]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
import torch

model_name = "facebook/mbart-large-50-many-to-many-mmt"
model = MBartForConditionalGeneration.from_pretrained(model_name)
tokenizer = MBart50TokenizerFast.from_pretrained(model_name)

In [14]:
# Translation Test
src_lang = "en_XX"
tgt_lang = "zh_CN"

# Func. def. if using GPU
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
# device = torch.device("cpu")
model = model.to(device)

def translate(model, tokenizer, sentence, src_lang, tgt_lang):
    tokenizer.src_lang = src_lang
    tokenizer.tgt_lang = tgt_lang
    encoded_input = tokenizer(sentence, return_tensors="pt").to(device)
    generated_tokens = model.generate(**encoded_input, forced_bos_token_id=tokenizer.lang_code_to_id[tgt_lang])
    translation = tokenizer.decode(generated_tokens[0], skip_special_tokens=True)
    return translation

print(translate(model, tokenizer, "你好吗?", 'zh_CN', 'en_XX'))
print(translate(model, tokenizer, "Can you please work?", 'en_XX', 'zh_CN'))
print(translate(model, tokenizer, "आज मेरा जन्मदिन हे", 'hi_IN', 'en_XX'))

How are you?
你能工作吗?
Today is my birthday


In [5]:
!pip3 install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.6/106.6 kB 3.1 MB/s eta 0:00:00


In [ ]:
# NOTE: If you already ran this before, load the translations from a text file and get the BLEU Score

# with open('mbarttranslations.txt', 'r', encoding='utf-8') as f:
#     hyp = [line.strip() for line in f.readlines()]
# refs = [[row['zh']] for _, row in validation_df.iterrows()]
# import sacrebleu
# bleu_score = sacrebleu.corpus_bleu(hyp, refs, tokenize='zh')
# print(f"BLEU Score: {bleu_score.score}")

In [15]:
# Method to perform SacreBleu test on pretrained model (en to zh)
import sacrebleu
def translate_and_evaluate(df, model, tokenizer, src_lang, tgt_lang):
    if src_lang == "en":
      tokenizer.src_lang, tokenizer.tgt_lang = "en_XX", "zh_CN"
    else:
      tokenizer.src_lang, tokenizer.tgt_lang = "zh_CN", "en_XX"

    hyp = []
    refs = [[] for _ in range(len(df))]
    for idx, row in df.iterrows():
        # Translate the sentence
        encoded_input = tokenizer(row[src_lang], return_tensors="pt").to(device)
        generated_tokens = model.generate(**encoded_input, forced_bos_token_id=tokenizer.lang_code_to_id[tokenizer.tgt_lang])
        translation = tokenizer.decode(generated_tokens[0], skip_special_tokens=True)
        hyp.append(translation)

        # Prepare references
        refs[idx].append(row[tgt_lang])

    # Compute BLEU score
    if tgt_lang == "en":
      sacrebleu_without_smoothing = sacrebleu.corpus_bleu(hyp, refs)
    else:
      sacrebleu_without_smoothing = sacrebleu.corpus_bleu(hyp, refs, tokenize='zh')
    return hyp, sacrebleu_without_smoothing

In [ ]:
# Perform the translation and evaluation
translations, bleu_score = translate_and_evaluate(validation_df, model, tokenizer, 'en', 'zh')
print(f"BLEU Score: {bleu_score.score}")

BLEU Score: 28.317815064640417


In [ ]:
# Save translations to a file
with open('mbarttranslations.txt', 'w', encoding='utf-8') as f:
    for translation in translations:
        f.write(translation + '\n')

In [ ]:
# Perform the backward translation and evaluation
translations, bleu_score = translate_and_evaluate(validation_df, model, tokenizer, 'zh', 'en')
print(f"BLEU Score: {bleu_score.score}")

BLEU Score: 52.511563937151365


In [ ]:
# Save translations to a file
with open('backwardmbarttranslations.txt', 'w', encoding='utf-8') as f:
    for translation in translations:
        f.write(translation + '\n')

In [16]:
# Define self-consistency simply using cosine_similarity
# TODO: Test other metrics e.g. MSE, etc.
import torch
from torch.nn.functional import cosine_similarity

def self_consistency_loss_en_to_zh(original_text, model, tokenizer):
    # Translate to Chinese
    translated_text = translate(model, tokenizer, original_text, "en_XX", "zh_CN")

    # Translate back to English
    back_translated_text = translate(model, tokenizer, translated_text, "zh_CN", "en_XX")

    # Encoded texts
    encoded_input = tokenizer(original_text, return_tensors="pt").to(device)
    encoded_back_translated_input = tokenizer(back_translated_text, return_tensors="pt").to(device)

    # Calculate cosine similarity between original and back-translated text embeddings
    original_embedding = model.get_encoder()(**encoded_input).last_hidden_state.mean(dim=1)
    back_translated_embedding = model.get_encoder()(**encoded_back_translated_input).last_hidden_state.mean(dim=1)
    loss = 1 - cosine_similarity(original_embedding, back_translated_embedding)

    return loss

def self_consistency_loss_zh_to_en(original_text, model, tokenizer):
    # Translate to English
    translated_text = translate(model, tokenizer, original_text, "zh_CN", "en_XX")

    # Translate back to Chinese
    back_translated_text = translate(model, tokenizer, translated_text, "en_XX", "zh_CN")

    # Encoded texts
    encoded_input = tokenizer(original_text, return_tensors="pt").to(device)
    encoded_back_translated_input = tokenizer(back_translated_text, return_tensors="pt").to(device)

    # Calculate cosine similarity between original and back-translated text embeddings
    original_embedding = model.get_encoder()(**encoded_input).last_hidden_state.mean(dim=1)
    back_translated_embedding = model.get_encoder()(**encoded_back_translated_input).last_hidden_state.mean(dim=1)
    loss = 1 - cosine_similarity(original_embedding, back_translated_embedding)

    return loss

In [9]:
# # Checking the maximum length of the tokens
# en_token_len = [len(tokenizer.encode(text)) for text in train_df['en']]
# avg_en_token_len = sum(en_token_len) / len(en_token_len)
# print(f"Average English Token Length: {avg_en_token_len}")

# zh_token_len = [len(tokenizer.encode(text)) for text in train_df['zh']]
# avg_zh_token_len = sum(zh_token_len) / len(zh_token_len)
# print(f"Average Chinese Token Length: {avg_zh_token_len}")

In [17]:
train_df_subset = train_df.sample(n=160, random_state=2)
train_df_subset

en  \
170681  And I realized if I kept up the way that I was...   
58278                                   And so was humor.   
211707  It sounded like the setup to a joke, but I kne...   
20680      We don't want to get caught flat-footed again.   
79419      Thank you very much for looking and listening.   
...                                                   ...   
217645  And when I went to the refugee camp in Pakista...   
216440                                       It's a fact.   
31074             Finger, finger, finger, finger. Yellow.   
208341             Why can't we celebrate it differently?   
124189  An LED is a semiconductor. It's an electronic ...   

                                                  zh  
170681  并且我意识到，如果我继续我以前的方式， 我基本上很难做出什么影响 在我的职业生涯结束前。  
58278                                       所以幽默也相像。  
211707                         听起来像是在开玩笑， 可我知道她是认真的。  
20680                                我们不想再毫无准备惊慌失措了。  
79419                                     非常感谢大家的观赏！  
...                                              ...  
217645                      当我来到巴基斯坦的难民营时 那里有七百五十万难民  
216440                                         这是事实。  
31074                                手指，手指，手指…………黄色球  
208341                               为什么不能换一种方式来庆祝呢？  
124189                              LED是个半导体，是个电子器件．  

[160 rows x 2 columns]

In [18]:
from torch.utils.data import Dataset, DataLoader

# You can adjust
# 1. max_length
# 2. batch_size

class TranslationDataset_en_to_zh(Dataset):
    # Here I chose max length 32 because it is bigger than 25.4 and 22.6
    def __init__(self, dataframe, tokenizer, max_length=32):
        self.dataframe = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        english_text = self.dataframe.iloc[idx]['en']
        chinese_text = self.dataframe.iloc[idx]['zh']
        tokenized_input = self.tokenizer(english_text, return_tensors="pt", max_length=self.max_length, padding="max_length", truncation=True)
        tokenized_target = self.tokenizer(chinese_text, return_tensors="pt", max_length=self.max_length, padding="max_length", truncation=True)
        input_ids = tokenized_input.input_ids.squeeze()
        attention_mask = tokenized_input.attention_mask.squeeze()
        labels = tokenized_target.input_ids.squeeze()
        return {"input_ids": input_ids, "attention_mask": attention_mask, "labels": labels}

class TranslationDataset_zh_to_en(Dataset):
    # Here I chose max length 32 because it is bigger than 25.4 and 22.6
    def __init__(self, dataframe, tokenizer, max_length=32):
        self.dataframe = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        english_text = self.dataframe.iloc[idx]['en']
        chinese_text = self.dataframe.iloc[idx]['zh']
        tokenized_input = self.tokenizer(chinese_text, return_tensors="pt", max_length=self.max_length, padding="max_length", truncation=True)
        tokenized_target = self.tokenizer(english_text, return_tensors="pt", max_length=self.max_length, padding="max_length", truncation=True)
        input_ids = tokenized_input.input_ids.squeeze()
        attention_mask = tokenized_input.attention_mask.squeeze()
        labels = tokenized_target.input_ids.squeeze()
        return {"input_ids": input_ids, "attention_mask": attention_mask, "labels": labels}

# Create the dataset
dataset_en_to_zh = TranslationDataset_en_to_zh(train_df_subset, tokenizer)
dataset_zh_to_en = TranslationDataset_zh_to_en(train_df_subset, tokenizer)

# Create the DataLoader
dataloader_en_to_zh = DataLoader(dataset_en_to_zh, batch_size=16, shuffle=True, num_workers=4)
dataloader_zh_to_en = DataLoader(dataset_zh_to_en, batch_size=16, shuffle=True, num_workers=4)


In [19]:
import time
import os
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

def save_checkpoint(model, optimizer, epoch, batch_index, checkpoint_path):
    torch.save({
        'epoch': epoch,
        'batch_index': batch_index,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict()
    }, checkpoint_path)
    print(f"Checkpoint saved at epoch {epoch}, batch {batch_index}")

# Define the checkpoint path
# checkpoint_dir = "/content/drive/MyDrive/checkpoints"
# os.makedirs(checkpoint_dir, exist_ok=True)

# # Check if a checkpoint exists
# latest_checkpoint_path = os.path.join(checkpoint_dir, "latest_checkpoint.pt")
# if os.path.exists(latest_checkpoint_path):
#     # Load the checkpoint
#     checkpoint = torch.load(latest_checkpoint_path)
#     model.load_state_dict(checkpoint['model_state_dict'])
#     optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
#     start_epoch = checkpoint['epoch']
#     start_batch_index = checkpoint['batch_index'] + 1
#     print(f"Restarting from {start_epoch} and batch {start_batch_index}")
# else:
#     start_epoch = 0
#     start_batch_index = 0

start_epoch = 0
start_batch_index = 0

# Hyper-Parameters
num_epochs = 1
consistency_weight = 0.1 # Weight to self-consistency loss vs translation loss
num_batches = len(dataloader_en_to_zh)
print(f"Total number of batches: {num_batches}")

# Training Loop for forward
model.train()
start_time = time.time()
for epoch in range(num_epochs):
    start_time = time.time()
    batch_number = 0
    for batch_index, batch in enumerate(dataloader_en_to_zh, start=1):
        batch_number += 1
        if batch_index < start_batch_index:
            continue

        # Reset the gradients after each mini batch
        optimizer.zero_grad()

        # Move data to GPU
        print("Batch number: ", batch_number)
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)
        # input_ids = batch["input_ids"]
        # attention_mask = batch["attention_mask"]
        # labels = batch["labels"]

        # Forward pass
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        translation_loss = outputs.loss

        # Self-consistency loss
        consistency_losses = []
        for input_id in input_ids:
            text = tokenizer.decode(input_id, skip_special_tokens=True)
            print(text)
            loss = self_consistency_loss_en_to_zh(text, model, tokenizer).to(device)
            # loss = self_consistency_loss(text, model, tokenizer)
            consistency_losses.append(loss)
        consistency_loss = torch.mean(torch.stack(consistency_losses))

        # Total loss
        total_loss = translation_loss + consistency_weight * consistency_loss

        # Backward pass and optimization
        total_loss.backward()
        optimizer.step()

        # if batch_index % 100 == 0:
        #     checkpoint_path = os.path.join(checkpoint_dir, f"checkpoint_epoch_{epoch}_batch_{batch_index}.pt")
        #     save_checkpoint(model, optimizer, epoch, batch_index, checkpoint_path)
        #     # Update the latest checkpoint path
        #     save_checkpoint(model, optimizer, epoch, batch_index, latest_checkpoint_path)

        #     # Time:
        #     curr_time = time.time()
        #     remaining_duration = ((curr_time - start_time) / (batch_index)) * (num_batches - batch_index)
        #     print(f"Batch number {batch_index}: Epoch {epoch}, Batch Loss: {total_loss.item()}, Estimated Time Left: {remaining_duration}")

    # Reset the start_batch_index after completing an epoch
    start_batch_index = 0

    print(f"Epoch {epoch + 1} completed for forward")

# Training Loop for backward

# Hyper-Parameters
num_epochs = 1
consistency_weight = 0.1 # Weight to self-consistency loss vs translation loss
num_batches = len(dataloader_zh_to_en)
print(f"Total number of batches: {num_batches}")

for epoch in range(num_epochs):
    start_time = time.time()
    batch_number = 0
    for batch_index, batch in enumerate(dataloader_zh_to_en, start=1):
        batch_number += 1
        if batch_index < start_batch_index:
            continue

        # Reset the gradients after each mini batch
        optimizer.zero_grad()

        # Move data to GPU
        print("Batch number: ", batch_number)
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)
        # input_ids = batch["input_ids"]
        # attention_mask = batch["attention_mask"]
        # labels = batch["labels"]

        # Forward pass
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        translation_loss = outputs.loss

        # Self-consistency loss
        consistency_losses = []
        for input_id in input_ids:
            text = tokenizer.decode(input_id, skip_special_tokens=True)
            print(text)
            loss = self_consistency_loss_zh_to_en(text, model, tokenizer).to(device)
            # loss = self_consistency_loss(text, model, tokenizer)
            consistency_losses.append(loss)
        consistency_loss = torch.mean(torch.stack(consistency_losses))

        # Total loss
        total_loss = translation_loss + consistency_weight * consistency_loss

        # Backward pass and optimization
        total_loss.backward()
        optimizer.step()

        # if batch_index % 100 == 0:
        #     checkpoint_path = os.path.join(checkpoint_dir, f"checkpoint_epoch_{epoch}_batch_{batch_index}.pt")
        #     save_checkpoint(model, optimizer, epoch, batch_index, checkpoint_path)
        #     # Update the latest checkpoint path
        #     save_checkpoint(model, optimizer, epoch, batch_index, latest_checkpoint_path)

        #     # Time:
        #     curr_time = time.time()
        #     remaining_duration = ((curr_time - start_time) / (batch_index)) * (num_batches - batch_index)
        #     print(f"Batch number {batch_index}: Epoch {epoch}, Batch Loss: {total_loss.item()}, Estimated Time Left: {remaining_duration}")

    # Reset the start_batch_index after completing an epoch
    start_batch_index = 0

    print(f"Epoch {epoch + 1} completed")


Total number of batches: 10
Batch number:  1
That's a pretty good -- but there's some of you out there that won't admit that you're makers.
How does it do it?
KB: Yeah.
[Allegiance] [Very intimate] [Serious mistakes] And tattoos tell you a lot of stories.
Okay, here we go.
Thank you very much for looking and listening.
So this is something that we're now going to try for the first time, live, and this is really, truly, very frightening.
And it turns out that the insects flap their wings in a very clever way, at a very high angle of attack that create
My passions are music, technology and making things.
What do we do now?
Why can't we celebrate it differently?
"A smile is a door that is half open and half closed," the author Jennifer Egan wrote.
There's a truck on the right. Boxes fall out of the truck.
So you can open this one.
There's also been a big paradigm shift in crime.
They talked about science and the need for a new scientific revolution.
Batch number:  2
And, as Marvin said, a

In [20]:
# Save the Model
model_dir = "/content/drive/MyDrive/mbart_model"
os.makedirs(model_dir, exist_ok=True)
torch.save(model.state_dict(), "/content/drive/MyDrive/mbart_model/mbart_translation_model_with_self_consistency_fwd_and_bkwd.pth")

In [21]:
# Load the Model
model_path = "/content/drive/MyDrive/mbart_model/mbart_translation_model_with_self_consistency_fwd_and_bkwd.pth"
model.load_state_dict(torch.load(model_path))
model.eval()

MBartForConditionalGeneration(
  (model): MBartModel(
    (shared): Embedding(250054, 1024, padding_idx=1)
    (encoder): MBartEncoder(
      (embed_tokens): Embedding(250054, 1024, padding_idx=1)
      (embed_positions): MBartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x MBartEncoderLayer(
          (self_attn): MBartAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): ReLU()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm): LayerNor

In [22]:
# TODO: Test the Model
import sacrebleu

translations, bleu_score = translate_and_evaluate(validation_df, model, tokenizer, 'en', 'zh')
print(f"BLEU Score: {bleu_score.score}")

# # Save translations to a file
# with open('tunedforwardmbarttranslations.txt', 'w', encoding='utf-8') as f:
#     for translation in translations:
#         f.write(translation + '\n')

BLEU Score: 24.6189261896478


Using a random sample of n = 160 from train_df, with seed = 2, bleu score when testing on validation df went from 28.317815 to 28.7253707.

zh -> en translation showed no error like when MBART was finetuned using 4000 batches, however backward translation and evaluation showed a decreased bleu score of 46.575793908279884 from 52.511563937151365

translations:
你好吗? -> How are you?
Can you please work? -> 你能工作吗?
आज मेरा जन्मदिन हे -> Today is my birthday

In [8]:
def translate(model, tokenizer, sentence, src_lang, tgt_lang):
    tokenizer.src_lang = src_lang
    tokenizer.tgt_lang = tgt_lang
    encoded_input = tokenizer(sentence, return_tensors="pt").to(device)
    generated_tokens = model.generate(**encoded_input, forced_bos_token_id=tokenizer.lang_code_to_id[tgt_lang])
    translation = tokenizer.decode(generated_tokens[0], skip_special_tokens=True)
    return translation

print(translate(model, tokenizer, "你好吗?", 'zh_CN', 'en_XX'))
print(translate(model, tokenizer, "Can you please work?", 'en_XX', 'zh_CN'))
print(translate(model, tokenizer, "आज मेरा जन्मदिन हे", 'hi_IN', 'en_XX'))

How are you?
你能工作吗?
Today is my birthday


In [23]:
# Testing backward translation
# Perform the translation and evaluation
translations, bleu_score = translate_and_evaluate(validation_df, model, tokenizer, 'zh', 'en')
print(f"BLEU Score: {bleu_score.score}")

BLEU Score: 46.575793908279884
